# Test UR Robot

In [ ]:
import rtde_control
import rtde_receive
import numpy as np
import yaml
import zmq
from interface.protobuf import robot_pb2

# Set up the RTDE connection
with open("config/address.yaml", "r") as f:
    robot_ip = yaml.load(f.read(), Loader=yaml.Loader)["robot_ip"]
rtde_r = rtde_receive.RTDEReceiveInterface(robot_ip)
rtde_c = rtde_control.RTDEControlInterface(robot_ip)

In [ ]:
class RobotPublisher:
    def __init__(self):
        self.context = zmq.Context()
        self.socket = self.context.socket(zmq.PUB)
        self.socket.bind("tcp://*:5555")
        
        self.robot = robot_pb2.Robot()

    def publish_robot_data(self, joint_angles: list, joint_velocities: list, tcp_pose: list):
        self.robot.joint_angles[:] = joint_angles
        self.robot.joint_velocities[:] = joint_velocities
        self.robot.tcp_pose[:] = tcp_pose
        
        self.socket.send(self.robot.SerializeToString())

## Robot State Visualization

This part is to visualize the robot state when human teaches the robot.

In [ ]:
def read_robot_data(rtde_r):
    joint_angles = rtde_r.getActualQ()
    joint_velocities = rtde_r.getActualQd()
    tcp_pose = rtde_r.getActualTCPPose()
    return joint_angles, joint_velocities, tcp_pose

def main():
    robot_publisher = RobotPublisher()
    try:
        while True:
            joint_angles, joint_velocities, tcp_pose = read_robot_data(rtde_r)
            robot_publisher.publish_robot_data(joint_angles, joint_velocities, tcp_pose)
    except KeyboardInterrupt:
        pass

## Move to Point

In [ ]:
target_point = np.array([0.5, 0.5, 0.5, 0, 0, 0])
move_vel = 0.1
move_acc = 0.3
rtde_c.moveL(target_point, move_vel, move_acc, False)